In [1]:
import pandas as pd
import numpy as np
import itertools
import datetime
import pandas_gbq
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
# %load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
project_id = "perceptive-ivy-290216"

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
query2=f"""
SELECT * FROM `perceptive-ivy-290216.f1_api.race_lap_time`
# WHERE YEAR=2024
# AND GP='Saudi Arabian Grand Prix'
"""
track3=pandas_gbq.read_gbq(query2,project_id,dialect='standard')

Downloading: 100%|██████████|


In [4]:
query_all_laps=f"""
SELECT * FROM `perceptive-ivy-290216.f1_api.race_lap_time`
UNION ALL
SELECT * FROM `perceptive-ivy-290216.f1_api.final_driver_positions_all`
"""
track_all_laps=pandas_gbq.read_gbq(query_all_laps,project_id,dialect='standard')

Downloading: 100%|██████████|


In [14]:
race="Italian Grand Prix"
years=2024

In [15]:
track2=track3[(track3["GP"]==race)&(track3["Year"]==years)]

In [16]:
track_all_laps[track_all_laps["Driver"]=="BOT"].tail()

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP
467235,0 days 02:35:34.118000,BOT,77,NaT,46.0,6.0,0 days 02:32:10.096000,NaT,NaT,0 days 00:00:39.168000,NaT,NaT,0 days 02:33:50.005000,NaT,102.0,158.0,104.0,NaN,False,SOFT,3.0,False,Mercedes,0 days 02:32:00.029000,NaT,1,3.0,False,,False,False,2020,Tuscan Grand Prix
467236,0 days 02:34:01.691000,BOT,77,0 days 00:01:42.702000,64.0,4.0,0 days 02:32:39.079000,NaT,0 days 00:00:48.698000,0 days 00:00:32.631000,0 days 00:00:21.373000,0 days 02:33:07.708000,0 days 02:33:40.339000,0 days 02:34:01.712000,279.0,277.0,254.0,308.0,False,SOFT,4.0,False,Mercedes,0 days 02:32:18.989000,NaT,1,14.0,False,,False,False,2021,Mexico City Grand Prix
467237,0 days 02:39:57.857000,BOT,77,0 days 00:01:49.327000,68.0,5.0,0 days 02:38:32.377000,NaT,0 days 00:00:52.425000,0 days 00:00:34.072000,0 days 00:00:22.830000,0 days 02:39:00.976000,0 days 02:39:35.048000,0 days 02:39:57.878000,290.0,200.0,256.0,310.0,False,SOFT,4.0,False,Mercedes,0 days 02:38:08.530000,NaT,1,16.0,False,,False,False,2021,Mexico City Grand Prix
467238,0 days 03:17:09.743000,BOT,77,0 days 00:02:15.332000,31.0,3.0,0 days 03:14:55.110000,NaT,0 days 00:00:31.974000,0 days 00:01:05.459000,0 days 00:00:37.899000,0 days 03:15:26.370000,0 days 03:16:31.829000,0 days 03:17:09.728000,131.0,143.0,151.0,127.0,False,HARD,11.0,False,Alfa Romeo,0 days 03:14:54.411000,NaT,1,10.0,False,,False,False,2022,Monaco Grand Prix
467239,0 days 03:35:02.121000,BOT,77,NaT,58.0,7.0,0 days 03:32:29.275000,NaT,0 days 00:00:48.390000,0 days 00:00:50.379000,0 days 00:00:56.046000,0 days 03:33:15.696000,0 days 03:34:06.071000,0 days 03:35:02.148000,117.0,118.0,288.0,200.0,False,SOFT,5.0,False,Alfa Romeo,0 days 03:31:58.635000,NaT,1,12.0,False,,False,False,2023,Australian Grand Prix


In [17]:
#Get Final Pos to order the final plot
track_all_laps_2=track_all_laps[(track_all_laps["GP"]==race)&(track_all_laps["Year"]==years)]
final_pos=track_all_laps_2[track_all_laps_2["LapNumber"]==track_all_laps_2.LapNumber.max()][["Driver","Position"]]

In [18]:
year=track2['Year'].iloc[0]
gp=track2['GP'].iloc[0]
Drivers=(track2['Driver'].unique())

In [19]:
stints = track2[["Driver", "Stint", "Compound", "LapNumber"]]
stints = stints.groupby(["Driver", "Stint", "Compound"]).count().reset_index()
stints = stints.rename(columns={"LapNumber": "StintLength"})
stints.head()

,Driver,Stint,Compound,StintLength
0,ALB,1.0,MEDIUM,17
1,ALB,2.0,HARD,36
2,ALO,1.0,MEDIUM,12
3,ALO,2.0,HARD,23
4,ALO,3.0,HARD,18


In [20]:
stint_with_pos=stints.merge(final_pos, left_on=["Driver"],right_on=["Driver"]).sort_values(by=['Position','Stint'])
stint_with_pos.head()

,Driver,Stint,Compound,StintLength,Position
18,LEC,1.0,MEDIUM,15,1.0
19,LEC,2.0,HARD,38,1.0
30,PIA,1.0,MEDIUM,16,2.0
31,PIA,2.0,HARD,22,2.0
32,PIA,3.0,HARD,15,2.0


In [21]:
fig=px.bar(
    stint_with_pos,
    y="Driver",
    x='StintLength',
    text="StintLength",
    orientation='h',
    barmode='stack',
    color='Compound',
    template="simple_white",
    title="<b>Tyre Strategies for the {} {}</b>".format(year,gp),
    height=800, width=1200,
    hover_data=["Position"],
    color_discrete_map={
                 "SOFT": "#F91536",
                 "MEDIUM": "#ffdf00 ",
                 "HARD":"#c8c8c8 ",
                 "INTERMEDIATE":"#009E60 ",
                 "WET":"#3671C6"
    }
)
fig.update_layout(
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    xaxis_title="<b>Lap Number</b>",
    yaxis_title="<b>Driver</b>",
    title_x=0.5,
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)

fig.update_layout(yaxis={'categoryorder':'total descending'},    
        hoverlabel=dict(
        bgcolor="Black",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    margin=dict(l=5, r=5, t=35, b=5),
    )

fig['layout']['yaxis']['autorange'] = "reversed"

fig.update_traces(marker_line_width=1, marker_line_color="WHITE", opacity=0.99, insidetextanchor="middle",textfont_color="White")

fig.update_yaxes(ticksuffix = "  ")

fig.show()

In [22]:
fig.write_html("/Users/rdesh723/statpulse-html/plots/Race Tyre Strat/{}/{} {} Tyre Strategies.html".format(year,year,gp),full_html=False, include_plotlyjs='cdn')

In [ ]:
#Multi Tracks

In [14]:
#2024
# race_multi=["Bahrain Grand Prix", "Saudi Arabian Grand Prix", "Australian Grand Prix", "Japanese Grand Prix","Chinese Grand Prix","Miami Grand Prix","Emilia Romagna Grand Prix","Monaco Grand Prix","Canadian Grand Prix","Spanish Grand Prix","Austrian Grand Prix","British Grand Prix","Hungarian Grand Prix","Belgian Grand Prix"]
#2023
# race_multi=["Abu Dhabi Grand Prix","Australian Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Canadian Grand Prix","Dutch Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Japanese Grand Prix","Las Vegas Grand Prix","Mexico City Grand Prix","Miami Grand Prix","Monaco Grand Prix","Pre-Season Testing","Qatar Grand Prix","Saudi Arabian Grand Prix","Singapore Grand Prix","Spanish Grand Prix","São Paulo Grand Prix","United States Grand Prix"]
#2022
# race_multi=["Abu Dhabi Grand Prix","Australian Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Canadian Grand Prix","Dutch Grand Prix","Emilia Romagna Grand Prix","French Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Japanese Grand Prix","Mexico City Grand Prix","Miami Grand Prix","Monaco Grand Prix","Saudi Arabian Grand Prix","Singapore Grand Prix","Spanish Grand Prix","São Paulo Grand Prix","United States Grand Prix"]
#2021
# race_multi=["Abu Dhabi Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Dutch Grand Prix","Emilia Romagna Grand Prix","French Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Mexico City Grand Prix","Monaco Grand Prix","Portuguese Grand Prix","Qatar Grand Prix","Russian Grand Prix","Saudi Arabian Grand Prix","Spanish Grand Prix","Styrian Grand Prix","São Paulo Grand Prix","Turkish Grand Prix","United States Grand Prix"]
#2020
race_multi=["70th Anniversary Grand Prix","Abu Dhabi Grand Prix","Austrian Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Eifel Grand Prix","Emilia Romagna Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Portuguese Grand Prix","Russian Grand Prix","Sakhir Grand Prix","Spanish Grand Prix","Styrian Grand Prix","Turkish Grand Prix","Tuscan Grand Prix"]

year_multi=2020
for i in race_multi:
    track2=track3[(track3["GP"]==i)&(track3["Year"]==year_multi)]
    track_all_laps_2=track_all_laps[(track_all_laps["GP"]==i)&(track_all_laps["Year"]==year_multi)]
    final_pos=track_all_laps_2[track_all_laps_2["LapNumber"]==track_all_laps_2.LapNumber.max()][["Driver","Position"]]
    year=track2['Year'].iloc[0]
    gp=track2['GP'].iloc[0]
    Drivers=(track2['Driver'].unique())
    stints = track2[["Driver", "Stint", "Compound", "LapNumber"]]
    stints = stints.groupby(["Driver", "Stint", "Compound"]).count().reset_index()
    stints = stints.rename(columns={"LapNumber": "StintLength"})
    stints.head()
    stint_with_pos=stints.merge(final_pos, left_on=["Driver"],right_on=["Driver"]).sort_values(by=['Position','Stint'])
    stint_with_pos.head()
    fig=px.bar(
        stint_with_pos,
        y="Driver",
        x='StintLength',
        text="StintLength",
        orientation='h',
        barmode='stack',
        color='Compound',
        template="simple_white",
        title="<b>Tyre Strategies for the {} {}</b>".format(year_multi,i),
        height=800, width=1200,
        hover_data=["Position"],
        color_discrete_map={
                    "SOFT": "#F91536",
                    "MEDIUM": "#ffdf00 ",
                    "HARD":"#c8c8c8 ",
                    "INTERMEDIATE":"#009E60 ",
                    "WET":"#3671C6"
        }
    )
    fig.update_layout(
        yaxis = dict(tickfont = dict(size=20)),
        xaxis = dict(tickfont = dict(size=20)),
        xaxis_title="<b>Lap Number</b>",
        yaxis_title="<b>Driver</b>",
        title_x=0.5,
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )

    fig.update_layout(yaxis={'categoryorder':'total descending'},    
            hoverlabel=dict(
            bgcolor="Black",
            font_size=16,
            font_family="PT Sans Narrow"
        ),
        margin=dict(l=5, r=5, t=35, b=5),
        )

    fig['layout']['yaxis']['autorange'] = "reversed"

    fig.update_traces(marker_line_width=1, marker_line_color="WHITE", opacity=0.99, insidetextanchor="middle",textfont_color="White")

    fig.update_yaxes(ticksuffix = "  ")
    fig.write_html("/Users/rdesh723/statpulse-html/plots/Race Tyre Strat/{}/{} {} Tyre Strategies.html".format(year_multi,year_multi,i),full_html=False, include_plotlyjs='cdn')
